## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-26-22-49-43 +0000,nypost,These two words led to federal indictment of J...,https://nypost.com/2025/09/26/us-news/these-tw...
1,2025-09-26-22-45-10 +0000,nypost,NYC Korean restaurant with $395 tasting menu t...,https://nypost.com/2025/09/26/lifestyle/nyc-ko...
2,2025-09-26-22-41-00 +0000,wsj,What to Know About the Criminal Case Against J...,https://www.wsj.com/us-news/law/what-to-know-a...
3,2025-09-26-22-39-19 +0000,nypost,‘Ultra rare’ 1 in 30 million orange lobster re...,https://nypost.com/2025/09/26/us-news/ultra-ra...
4,2025-09-26-22-36-32 +0000,latimes,"Pet hotel to pay $150,000 to settle allegation...",https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
76,trump,66
6,comey,27
345,new,23
13,will,20
131,tariffs,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
323,2025-09-25-23-56-17 +0000,nypost,Trump celebrates James Comey indictment in sca...,https://nypost.com/2025/09/25/us-news/trump-ce...,151
112,2025-09-26-18-49-37 +0000,nypost,Trump predicts ‘there will be others’ after Ja...,https://nypost.com/2025/09/26/us-news/trump-pr...,150
236,2025-09-26-10-20-21 +0000,nyt,"James Comey, Former FBI Director, Indicted Aft...",https://www.nytimes.com/2025/09/25/us/politics...,135
157,2025-09-26-16-22-19 +0000,bbc,Watch: Trump says 'there will be others' after...,https://www.bbc.com/news/videos/ceq2zzp1dx2o?a...,132
69,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...,131


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
323,151,2025-09-25-23-56-17 +0000,nypost,Trump celebrates James Comey indictment in sca...,https://nypost.com/2025/09/25/us-news/trump-ce...
229,82,2025-09-26-10-56-58 +0000,nypost,Netanyahu took unusual long flight to NY to gi...,https://nypost.com/2025/09/26/world-news/netan...
69,65,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...
310,54,2025-09-26-01-00-00 +0000,wsj,The Supreme Court is poised to decide whether ...,https://www.wsj.com/economy/central-banking/fe...
246,52,2025-09-26-10-00-00 +0000,latimes,A U.S. veteran spoke out against his wrongful ...,https://www.latimes.com/california/story/2025-...
43,45,2025-09-26-21-25-31 +0000,nyt,Sinclair and Nexstar End Jimmy Kimmel Boycott ...,https://www.nytimes.com/2025/09/26/business/me...
232,36,2025-09-26-10-32-37 +0000,nypost,Iran and Russia sign $25 billion agreement to ...,https://nypost.com/2025/09/26/world-news/iran-...
116,36,2025-09-26-18-30-06 +0000,nypost,Who is Howard Rubin? The ex-NYC financier with...,https://nypost.com/us-news/who-is-howard-rubin...
324,34,2025-09-25-23-56-00 +0000,wsj,Secretary of Defense Pete Hegseth has ordered ...,https://www.wsj.com/politics/national-security...
288,32,2025-09-26-04-00-00 +0000,wsj,The Trump administration is weighing a new pla...,https://www.wsj.com/economy/trade/trump-chip-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
